In [1]:
#import extensions
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
import time as time



In [2]:
#load Imag Data for delamination
DelamDataNoFailure = pd.read_csv('Z:\\HMS\\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Train_Imag_Data_Delamination.csv')
DelamDataLowFailure = pd.read_csv('Z:\\HMS\\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Failure_data_Delam_low_imag.csv')
DelamDataHighFailure = pd.read_csv('Z:\\HMS\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Failure_data_Delam_high_imag.csv')

#load Imag Data for reduction of membrane thickness
RedThicknessDataNoFailure = pd.read_csv('Z:\\HMS\\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Train_Imag_Data_RedThickness.csv')
RedThicknessDataLowFailure = pd.read_csv('Z:\\HMS\\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Failure_data_RedThickness_low_img.csv')
RedThicknessDataHighFailure = pd.read_csv('Z:\\HMS\\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Failure_data_RedThickness_high_imag.csv')

#rint(DelamDataNoFailure)

<>:4: SyntaxWarning: invalid escape sequence '\F'
<>:4: SyntaxWarning: invalid escape sequence '\F'
C:\Users\leonb\AppData\Local\Temp\ipykernel_13888\2589247029.py:4: SyntaxWarning: invalid escape sequence '\F'
  DelamDataHighFailure = pd.read_csv('Z:\\HMS\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Failure_data_Delam_high_imag.csv')


In [3]:
#add noise to imag data (and save it to csv file) 
#10 instances of each operating point (for each row)

#define function to add noise to the data 
def add_noise(data, num_variations=10, noise_level=0.001):
    noisy_data = []
    for _, row in data.iterrows():  # "_," is used to ignore the index of the row
        for _ in range(num_variations):
            # Create noise only for columns after the third column
            noise = np.random.normal(0, noise_level, size=row.iloc[3:].shape)
            noisy_row = row.copy()
            noisy_row.iloc[3:] += noise  # Add noise only to columns 4 and beyond
            noisy_data.append(noisy_row)
    return pd.DataFrame(noisy_data, columns=data.columns)

## add noise to the data
#Delamination
DelamDataNoFailure_noisy = add_noise(DelamDataNoFailure, num_variations=10, noise_level=0.001)
DelamDataLowFailure_noisy = add_noise(DelamDataLowFailure, num_variations=10, noise_level=0.001) 
DelamDataHighFailure_noisy = add_noise(DelamDataHighFailure, num_variations=10, noise_level=0.001)   

#RedThickness
#RedThicknessDataNoFailure_noisy = add_noise(RedThicknessDataNoFailure, num_variations=10, noise_level=0.001)
RedThicknessDataLowFailure_noisy = add_noise(RedThicknessDataLowFailure, num_variations=10, noise_level=0.001)
RedThicknessDataHighFailure_noisy = add_noise(RedThicknessDataHighFailure, num_variations=10, noise_level=0.001)

## mix noise data and non noise data randomly rowwise 
#Delamination    
DelamDataNoFailure_combined = pd.concat([DelamDataNoFailure, DelamDataNoFailure_noisy], ignore_index=True)  
DelamDataLowFailure_combined = pd.concat([DelamDataLowFailure, DelamDataLowFailure_noisy], ignore_index=True)
DelamDataHighFailure_combined = pd.concat([DelamDataHighFailure, DelamDataHighFailure_noisy], ignore_index=True)

#RedThickness
#RedThicknessDataNoFailure_combined = pd.concat([RedThicknessDataNoFailure, RedThicknessDataNoFailure_noisy], ignore_index=True)
RedThicknessDataLowFailure_combined = pd.concat([RedThicknessDataLowFailure, RedThicknessDataLowFailure_noisy], ignore_index=True)
RedThicknessDataHighFailure_combined = pd.concat([RedThicknessDataHighFailure, RedThicknessDataHighFailure_noisy], ignore_index=True)


##shuffle the data randomly
#Delamination
DelamDataNoFailure_shuffeled = shuffle(DelamDataNoFailure_combined, random_state=42) #shuffle the data randomly; random state to make shuffle reproducable
DelamDataLowFailure_shuffeled = shuffle(DelamDataLowFailure_combined, random_state=42)
DelamDataHighFailure_shuffeled = shuffle(DelamDataHighFailure_combined, random_state=42)

##RedThickness
#RedThicknessDataNoFailure_shuffeled = shuffle(RedThicknessDataNoFailure_combined, random_state=42)
RedThicknessDataLowFailure_shuffeled = shuffle(RedThicknessDataLowFailure_combined, random_state=42)
RedThicknessDataHighFailure_shuffeled = shuffle(RedThicknessDataHighFailure_combined, random_state=42)

#save shuffeled data to csv file for examination
#DelamDataNoFailure_shuffeled.to_csv('Z:\\HMS\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Train_Imag_Data_Delamination_shuffeled.csv', index=False)
#DelamDataLowFailure_shuffeled.to_csv('Z:\\HMS\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Failure_data_Delam_low_imag_shuffeled.csv', index=False)
#DelamDataHighFailure_shuffeled.to_csv('Z:\\HMS\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Failure_data_Delam_high_imag_shuffeled.csv', index=False)





In [4]:
##lable data 
#Delamination
DelamDataNoFailure_labeled = DelamDataNoFailure_shuffeled.copy()
DelamDataNoFailure_labeled['Label'] = 'DelamNoFailure'  # Label for no failure
DelamDataLowFailure_labeled = DelamDataLowFailure_shuffeled.copy()
DelamDataLowFailure_labeled['Label'] = 'DelamLowFailure'  # Label for low failure
DelamDataHighFailure_labeled = DelamDataHighFailure_shuffeled.copy()
DelamDataHighFailure_labeled['Label'] = 'DelamHighFailure'  # Label for high failure

#RedThickness
#RedThicknessDataNoFailure_labeled = RedThicknessDataNoFailure_shuffeled.copy()
#RedThicknessDataNoFailure_labeled['Label'] = 'RedThicknessNoFailure'  # Label for no failure # --> commented out since RedThicknessNoFailure == DelamNoFailure
RedThicknessDataLowFailure_labeled = RedThicknessDataLowFailure_shuffeled.copy()
RedThicknessDataLowFailure_labeled['Label'] = 'RedThicknessLowFailure'  # Label for low failure
RedThicknessDataHighFailure_labeled = RedThicknessDataHighFailure_shuffeled.copy()
RedThicknessDataHighFailure_labeled['Label'] = 'RedThicknessHighFailure'  # Label for high failure

#combine all data into one dataframe and shuffle it again
combined_data_Delam = pd.concat([DelamDataNoFailure_labeled, DelamDataLowFailure_labeled, DelamDataHighFailure_labeled], ignore_index=True)
combined_data_RedThickness = pd.concat([ RedThicknessDataLowFailure_labeled, RedThicknessDataHighFailure_labeled], ignore_index=True)
combined_data = pd.concat([combined_data_Delam, combined_data_RedThickness], ignore_index=True)  # Combine all data into one dataframe
combined_data = shuffle(combined_data, random_state=42)  # Shuffle the combined data
#print(combined_data)
#save combined data to csv file for examination
#combined_data.to_csv('Z:\\HMS\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\Train_Imag_Data_combined.csv', index=False)

In [5]:
#define test and train data for NN classification 
train_set, test_set = train_test_split(combined_data, test_size=0.2, random_state=42)  # 80% training, 20% testing
#print(train_set)
X_train = train_set.iloc[:,3:-1]
y_train = train_set['Label'].map({'DelamNoFailure': 0, 'DelamLowFailure': 1, 'DelamHighFailure': 2, 'RedThicknessLowFailure': 3, 'RedThicknessHighFailure': 4})  # Map labels to integers to use numerical values
print(X_train.shape, y_train.shape)
print(train_set.shape[1] - 4)


#find nan values in y_train (NaN values cause the NN to crash)
nan_count = y_train.isna().sum()
print(f"Number of NaN values in y_train: {nan_count}") 

(476212, 37) (476212,)
37
Number of NaN values in y_train: 0


In [6]:
#create Neural Network (defined in a function to prepare it for grid search)
tf.random.set_seed(42)  # Set random seed for reproducibility
def create_model(optimizer='sgd', activation='relu', hidden_layers=1, n_neuron=20):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(train_set.shape[1] - 4,)))  # Adjust input shape
    for _ in range(hidden_layers):
        model.add(tf.keras.layers.Dense(n_neuron, activation=activation))
    model.add(tf.keras.layers.Dense(5, activation='softmax')) # five output neurons with indiced of 0-4 for the five classes
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [12]:
#set up Grid search
param_grid = {
    'model__optimizer': ['sgd'],  # Optimizer
    'model__activation': ['relu'],  # Activation function
    'model__hidden_layers': [3,4],  # Number of hidden layers
    'model__n_neuron': [300],  # Number of neurons per layer
    'epochs': [100],  # Number of epochs
    
}

model = KerasClassifier(
    model=create_model,
    batch_size=40,  # Fixed batch size
    verbose=1  # Enable progress bar for each epoch
)

# Create the GridSearchCV object
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3,
    verbose=2,  # Show progress for each fold
    n_jobs = -1  # Use all available cores
)

# Fit the grid search to the training data
# Fit the grid search to the training data
start_time = time.time()
print("Starting grid search...")
grid_search.fit(
    X_train,
    y_train,
)
print("Grid search completed in {:.2f} seconds.".format(time.time() - start_time))

Starting grid search...
Fitting 3 folds for each of 2 candidates, totalling 6 fits


KeyboardInterrupt: 

In [9]:
#export results to csv file 
results_df = pd.DataFrame(grid_search.cv_results_)
results_df.to_csv('Z:\\HMS\\Fehlerdatenbasis\\EEC_Parameter_Programm\\Train_Data\\grid_search_results.csv', index=False)
print("Grid search results saved to CSV file.")

Grid search results saved to CSV file.
